# Ensuring different transaction codes have the same economic impact

In this training module we'll see how to use LUSID to perform the following task:

**<div align="center">As a data controller, I load transactions from two providers that use a different transaction code to signal the same economic activity. I want to ensure that LUSID applies a uniform economic impact when generating holdings.</div>**

In [1]:
# Set up LUSID
import os
import pandas as pd
import json
import uuid
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import logging
logging.basicConfig(level=logging.INFO)

import lusid as lu
import lusid.api as la
import lusid.models as lm

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.lpt.lpt import to_date

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token = RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name = "LusidJupyterNotebook"
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.9396.0,0.5.2887,"{'relation': 'RequestLogs', 'href': 'http://ja..."


In [2]:
# Create a scope and code to segregate data in this module from other modules
module_scope = "FBNUniversity"
module_code = "Module-6-1"
print(f"'{module_scope}/{module_code}' scope and code created.")

'FBNUniversity\Module-6-1' scope and code created.


## 1. Examining the source file

Imagine we load transactions into LUSID from both Societe Generale and Deutsche Bank:

* SG uses a transaction code of `Acheter` to signal an equity purchase.
* DB uses a transaction code of `Kaufen` to signal an equity purchase.

We want to retain these codes in the imported data but ensure LUSID applies the same economic impact to every transaction when generating holdings.

In [3]:
# Read transactions from amalgamated stream into pandas dataframe
sg_db_df = pd.read_csv("data/6-1-sg-db.csv", keep_default_na = False)
sg_db_df.index += 1
display(sg_db_df)

,source,instrument,asset,figi,txn_id,txn_code,trade_date,settle_date,units,price,currency
1,Lusid,GBP,Cash,,61001,FundsIn,2022-06-06T09:00:00Z,2022-06-08T09:00:00Z,500,1,GBP
2,SocieteGenerale,BP,Equity,BBG000C05BD1,61002,Acheter,2022-06-06T10:00:00Z,2022-06-08T09:00:00Z,50,5,GBP
3,DeutscheBank,BP,Equity,BBG000C05BD1,61003,Kaufen,2022-06-06T11:00:00Z,2022-06-08T09:00:00Z,25,5,GBP


## 2. Ensuring instruments are mastered correctly

It's possible the equity instrument in our transaction source file is already mastered in LUSID as part of the demonstration data, but for the avoidance of doubt we'll master it separately in a segregated custom instrument scope.

Note the `GBP` currency instrument, as per all currencies, is mastered out-of-the-box in the `default` instrument scope.

In [4]:
# Obtain the LUSID Instruments API
instruments_api = api_factory.build(la.InstrumentsApi)

# Create a dictionary of instrument definitions
definitions = {}

# Iterate over each row in the dataframe
for index, security in sg_db_df.iterrows():

    # Model equities
    if security["asset"] == "Equity":
        # Create definitions
        definitions[security["instrument"]] = lm.InstrumentDefinition(
            name = security["instrument"],
            identifiers = {
                "Figi": lm.InstrumentIdValue(value = security["figi"]),
            },
            definition = lm.Equity(
                instrument_type = "Equity",
                dom_ccy = security["currency"],
                identifiers = {}
            )
        )

# Upsert instruments to LUSID
upsert_instruments_response = instruments_api.upsert_instruments(
    request_body = definitions,
    # Master the instruments in a custom scope
    scope = f"{module_scope}{module_code}",
)

# Transform API response to a dataframe and show internally-generated unique LUID for each mastered instrument
upsert_instruments_response_df = lusid_response_to_data_frame(list(upsert_instruments_response.values.values()))
display(upsert_instruments_response_df[["name", "lusid_instrument_id"]])

,name,lusid_instrument_id
0,BP,LUID_00003DA0


## 3. Creating a suitable portfolio 
We register the custom instrument scope using the `instrument_scopes` field to ensure transactions upserted to the portfolio resolve first against instruments mastered in the custom scope, before falling back to the `default` scope.

In [5]:
# Obtain the LUSID Transaction Portfolio API
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)

# Create portfolio definition
portfolio_definition=lm.CreateTransactionPortfolioRequest(
    display_name="Training module 6.1 - Different transaction codes, same impact",
    code = module_code,
    base_currency = "GBP",
    # Must be before first transaction recorded
    created="2022-01-01",
    # Attempt to resolve transactions to instruments in the custom scope before falling back to the default scope
    instrument_scopes = [f"{module_scope}{module_code}"],
)

# Upsert portfolio to LUSID, making sure it's not already there
try:
    create_portfolio_response=transaction_portfolios_api.create_portfolio(
        scope = module_scope,
        create_transaction_portfolio_request = portfolio_definition
    )
    # Confirm success
    print(f"Portfolio with display name '{create_portfolio_response.display_name}' created effective {str(create_portfolio_response.created)}")
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "PortfolioWithIdAlreadyExists":
            logging.info(json.loads(e.body)["title"])

INFO:root:Could not create a portfolio with id 'Module-6-1' because it already exists in scope 'FBNUniversity'.


## 4. Configuring the built-in Buy transaction type

The built-in `Buy` transaction type is designed to increase an instrument holding by the number of units purchased, and decrease a cash holding by the total consideration.

We want `Acheter` and `Kaufen` to have the same economic impact.

### 4.1 Retrieving the definition of Buy to obtain the correct settings

We can call the `GetTransactionType` API to retrieve the definition of `Buy` before appending an alias for each of `Acheter` and `Kaufen`, to ensure we retain all the other settings correctly.

In [6]:
# Obtain the LUSID Transaction Type Configuration API
transaction_config_api = api_factory.build(la.TransactionConfigurationApi)

# Get existing transaction type definition    
try:
    response = transaction_config_api.get_transaction_type(
        # Specify the source of the transaction type
        source = "default",
        # Specify the primary alias name (in this case, the name of the only alias in the transaction type)
        type = "Buy",
    )
    display(response)
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "TransactionTypeNotFound":
        logging.info("Transaction type not found")

{'aliases': [{'description': 'Purchase',
              'is_default': False,
              'transaction_class': 'Basic',
              'transaction_roles': 'LongLonger',
              'type': 'Buy'},
             {'description': 'Purchase from Societe Generale',
              'is_default': False,
              'transaction_class': 'Basic',
              'transaction_roles': 'LongLonger',
              'type': 'Acheter'},
             {'description': 'Purchase from Deutsche Bank',
              'is_default': False,
              'transaction_class': 'Basic',
              'transaction_roles': 'LongLonger',
              'type': 'Kaufen'}],
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://jamleed.lusid.com/app/insights/logs/0HMI9H6G5GA1I:00000006',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'movements': [{'direction': 1,
                'mappings

### 4.2 Modifying the definition of Buy to append new aliases

We can now create a new definition for `Buy` that appends aliases for `Acheter` and `Kaufen` but retains all the other settings.

In [7]:
transaction_type_definition = lm.TransactionTypeRequest(
    aliases = [
        # Recreate the built-in Buy alias
        lm.TransactionTypeAlias(
            type = "Buy",
            description = "Purchase",
            transaction_class = "Basic",
            transaction_roles = "LongLonger"
        ),
        # Append a new alias for Acheter
        lm.TransactionTypeAlias(
            type = "Acheter",
            description = "Purchase from Societe Generale",
            transaction_class = "Basic",
            transaction_roles = "LongLonger"
        ),
        # Append a new alias for Kaufen
        lm.TransactionTypeAlias(
            type = "Kaufen",
            description = "Purchase from Deutsche Bank",
            transaction_class = "Basic",
            transaction_roles = "LongLonger"
        )
    ],
    # Recreate the built-in Buy movements
    movements = [
        lm.TransactionTypeMovement(
            movement_types = "StockMovement",
            side = "Side1",
            direction = 1,
        ),
        lm.TransactionTypeMovement(
            movement_types = "CashCommitment",
            side = "Side2",
            direction = -1,
        ),
    ]
)

# Replace transaction type in LUSID    
try:
    response = transaction_config_api.set_transaction_type(
        # Specify the source (data provider) of the transaction type (Buy is in the 'default' source)
        source = "default",
        # Specify the primary alias name
        type = "Buy",
        transaction_type_request = transaction_type_definition
    )
    # Confirm by retrieving the newly-modified transaction type
    display(transaction_config_api.get_transaction_type(source = "default", type = "Buy"))
except lu.ApiException as e:
    print(e)

{'aliases': [{'description': 'Purchase',
              'is_default': False,
              'transaction_class': 'Basic',
              'transaction_roles': 'LongLonger',
              'type': 'Buy'},
             {'description': 'Purchase from Societe Generale',
              'is_default': False,
              'transaction_class': 'Basic',
              'transaction_roles': 'LongLonger',
              'type': 'Acheter'},
             {'description': 'Purchase from Deutsche Bank',
              'is_default': False,
              'transaction_class': 'Basic',
              'transaction_roles': 'LongLonger',
              'type': 'Kaufen'}],
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://jamleed.lusid.com/app/insights/logs/0HMI9H6G5GA1I:00000008',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'movements': [{'direction': 1,
                'mappings

## 5. Loading transactions into the portfolio

We can now upsert transactions into LUSID, mapping the **txn_code** column in the source file to the `type` field, safe in the knowledge that the `Acheter` and `Kaufen` transaction types will have the same economic impact as `Buy`.

In [8]:
# Create list of transactions to upsert
transactions = []

# For each row in dataframe
for index, txn in sg_db_df.iterrows():

    # Set variables based on whether instrument is cash or not
    if txn["asset"] == "Cash":
        # Set the unique instrument identifier to be a currency
        identifiers = {"Instrument/default/Currency": txn["currency"]}
    else:
        # Set the unique instrument identifier to be a FIGI
        identifiers = {"Instrument/default/Figi": txn["figi"]}

    transactions.append(
        lm.TransactionRequest(
            transaction_id = txn["txn_id"],
            # Set the transaction type to the transaction code from the data provider
            type = txn["txn_code"],
            instrument_identifiers = identifiers,
            transaction_date = txn["trade_date"],
            settlement_date = txn["settle_date"],
            units = txn["units"],
            transaction_price = lm.TransactionPrice(price = txn["price"], type="Price"),
            total_consideration = lm.CurrencyAndAmount(
                # Calculate cost on-the-fly
                amount = txn["units"] * txn["price"],
                currency = txn["currency"]
            ),
        )
    )

# Upsert transactions to LUSID
upsert_transactions_response = transaction_portfolios_api.upsert_transactions(
    scope = module_scope, 
    code = module_code, 
    transaction_request = transactions
)

display(f"Transactions loaded at {str(upsert_transactions_response.version.as_at_date)}")
display(upsert_transactions_response)

'Transactions loaded at 2022-06-09 09:36:52.997370+00:00'

{'href': 'https://jamleed.lusid.com/api/api/transactionportfolios/FBNUniversity/Module-6-1/transactions?asAt=2022-06-09T09%3A36%3A52.9973700%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://jamleed.lusid.com/api/api/portfolios/FBNUniversity/Module-6-1?effectiveAt=2022-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2022-06-09T09%3A36%3A52.9973700%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://jamleed.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://jamleed.lusid.com/app/insights/logs/0HMI9H6G5GA1I:00000009',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'version': {'as_at_date': datetime.datetime(202

## 6. Generating holdings

### 6.1 On the trade date
We can now call the `GetHoldings` API for end-of-day on the trade date (6 June 2022) to see the interim impact of our transaction types on holdings.

In [9]:
# Get holdings for portfolio effective EOD 6 June 2022
get_holdings_response=transaction_portfolios_api.get_holdings(
    scope = module_scope, 
    code = module_code,
    # Specify an explicit date
    effective_at = "2022-06-06T17:00:00Z",
    # Decorate the instrument name property onto holdings to make the API response more intuitive
    property_keys=["Instrument/default/Name"],
)

# Transform API response to a pandas dataframe and show it
get_holdings_response_df=lusid_response_to_data_frame(get_holdings_response, rename_properties=True)
# Drop some noisy columns
get_holdings_response_df.drop(columns=[
    "sub_holding_keys", "cost_portfolio_ccy.currency", "currency", "SourcePortfolioId(default-Properties)", "SourcePortfolioScope(default-Properties)" ], inplace=True)
display(get_holdings_response_df)

,instrument_scope,instrument_uid,Name(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/Currency,transaction.instrument_scope,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties,transaction.entry_date_time,transaction.transaction_status,transaction.instrument_identifiers.Instrument/default/Figi,transaction.properties.Transaction/default/ResultantHolding.key,transaction.properties.Transaction/default/ResultantHolding.value.metric_value.value
0,FBNUniversityModule-6-1,LUID_00003DA0,BP,P,75.00,0.00,375.00,GBP,375.00,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,default,CCY_GBP,GBP,A,500.00,0.00,500.00,GBP,500.00,61001,FundsIn,GBP,default,CCY_GBP,2022-06-06 09:00:00+00:00,2022-06-08 09:00:00+00:00,500.00,1.00,Price,500.00,GBP,1.00,GBP,{},2022-06-09 09:46:51.478240+00:00,Active,NaN,NaN,NaN
2,default,CCY_GBP,GBP,C,-250.00,0.00,-250.00,GBP,-250.00,61002,Acheter,NaN,FBNUniversityModule-6-1,LUID_00003DA0,2022-06-06 10:00:00+00:00,2022-06-08 09:00:00+00:00,50.00,5.00,Price,250.00,GBP,1.00,GBP,NaN,2022-06-09 09:46:51.478240+00:00,Active,BBG000C05BD1,Transaction/default/ResultantHolding,50.00
3,default,CCY_GBP,GBP,C,-125.00,0.00,-125.00,GBP,-125.00,61003,Kaufen,NaN,FBNUniversityModule-6-1,LUID_00003DA0,2022-06-06 11:00:00+00:00,2022-06-08 09:00:00+00:00,25.00,5.00,Price,125.00,GBP,1.00,GBP,NaN,2022-06-09 11:05:15.611011+00:00,Active,BBG000C05BD1,Transaction/default/ResultantHolding,75.00


### 6.2 Post-settlement date

If we call `GetHoldings` API without an explicit date, LUSID uses today's date so we can see the impact on holdings after the settlement date.

In [10]:
# Get holdings for portfolio effective today
get_holdings_response=transaction_portfolios_api.get_holdings(
    scope = module_scope, 
    code = module_code,
    # Decorate the instrument name property onto holdings to make the API response more intuitive
    property_keys=["Instrument/default/Name"],
)

# Transform API response to a pandas dataframe and show it
get_holdings_response_df=lusid_response_to_data_frame(get_holdings_response, rename_properties=True)
# Drop some noisy columns
get_holdings_response_df.drop(columns=[
    "sub_holding_keys", "cost_portfolio_ccy.currency", "currency", "SourcePortfolioId(default-Properties)", "SourcePortfolioScope(default-Properties)"], inplace=True)
display(get_holdings_response_df)

,instrument_scope,instrument_uid,Name(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,FBNUniversityModule-6-1,LUID_00003DA0,BP,P,75.00,75.00,375.00,GBP,375.00
1,default,CCY_GBP,GBP,B,125.00,125.00,125.00,GBP,125.00
